In [17]:
### 기본 Skip-gram 클래스
from torch import nn

class VanilaSkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim  = embedding_dim
        )
        self.linear = nn.Linear(
            in_features  = embedding_dim,
            out_features = vocab_size
        )

    def forward(self, input_ids):
        embeddings = self.embedding(input_ids)
        output     = self.linear(embeddings)
        return output

In [3]:
### 영화 리뷰 데이터세트 전처리
import pandas as pd
from Korpora import Korpora
from konlpy.tag import Okt


corpus = Korpora.load("nsmc")
corpus = pd.DataFrame(corpus.test)  # 테스트세트 불러오기


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at /Users/seoyun/Korpora/nsmc/ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at /Users/seoyun

In [4]:
# 데이터세트 형태소 추출
tokenizer = Okt()
tokens    = [tokenizer.morphs(review) for review in corpus.text]
print(tokens[:3])

[['굳', 'ㅋ'], ['GDNTOPCLASSINTHECLUB'], ['뭐', '야', '이', '평점', '들', '은', '....', '나쁘진', '않지만', '10', '점', '짜', '리', '는', '더', '더욱', '아니잖아']]


In [9]:
### 단어 사전 구축
from collections import Counter


def build_vocab(corpus, n_vocab, special_tokens):
    counter = Counter()
    for tokens in corpus:
        counter.update(tokens)
    # 특수토큰 - <unk>: OOV에 대응하기 위한 토큰, 단어 사전 내에 없는 모든 단어 대체
    vocab = special_tokens
    # 가장 많이 등장한 토큰 순서로 사전 구축
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

# 토큰화된 데이터로 단어 사전 구축 (n_vocab: 구축할 단어사전의 크기)
vocab       = build_vocab(corpus=tokens, n_vocab=5000, special_tokens=["<unk>"])
token_to_id = {token: idx for idx, token in enumerate(vocab)}
id_to_token = {idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))   # 최대 길이 5000 + 특수 토큰 1 = 5001

['<unk>', '.', '이', '영화', '의', '..', '가', '에', '...', '을']
5001


In [10]:
### Skip-gram의 단어 쌍 추출

# 토큰을 입력받아 skip-gram의 학습 데이터로 사용하도록 전처리 - 1.(중심단어, 주변단어 쌍)
def get_word_pairs(tokens, window_size):
    pairs = []
    for sentence in tokens:
        sentence_length = len(sentence)
        for idx, center_word in enumerate(sentence):
            window_start  = max(0, idx - window_size)
            window_end    = min(sentence_length, idx + window_size + 1)
            center_word   = sentence[idx]
            context_words = sentence[window_start:idx] + sentence[idx+1:window_end]
            for context_word in context_words:
                pairs.append([center_word, context_word])
    return pairs


word_pairs = get_word_pairs(tokens, window_size=2)
print(word_pairs[:10])

[['굳', 'ㅋ'], ['ㅋ', '굳'], ['뭐', '야'], ['뭐', '이'], ['야', '뭐'], ['야', '이'], ['야', '평점'], ['이', '뭐'], ['이', '야'], ['이', '평점']]


In [11]:
### 인덱스 쌍 변환

# 토큰을 입력받아 skip-gram의 학습 데이터로 사용하도록 전처리 - 2.인덱스 쌍으로 변환
def get_index_pairs(word_pairs, token_to_id):
    pairs = []
    unk_index = token_to_id["<unk>"]
    for word_pair in word_pairs:
        center_word, context_word = word_pair
        # c.get(a, b): a가 c에 있으면 a의 인덱스, 없으면 b의 인덱스 반환
        # 단어사전에 있으면 해당 토큰의 인덱스 반환, 없으면 <unk> 인덱스 반환
        center_index  = token_to_id.get(center_word, unk_index)
        context_index = token_to_id.get(context_word, unk_index)
        pairs.append([center_index, context_index])
    return pairs


index_pairs = get_index_pairs(word_pairs, token_to_id)
print(index_pairs[:5])
print(len(vocab))

[[595, 100], [100, 595], [77, 176], [77, 2], [176, 77]]
5001


In [13]:
### 데이터로더 적용
import torch
from torch.utils.data import TensorDataset, DataLoader


index_pairs     = torch.tensor(index_pairs)
center_indexs   = index_pairs[:, 0]
contenxt_indexs = index_pairs[:, 1]

# 토큰을 입력받아 skip-gram의 학습 데이터로 사용하도록 전처리 - 3.텐서로 변환
dataset    = TensorDataset(center_indexs, contenxt_indexs)  # [N,2]구조
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

/var/folders/9y/9s104_6d69g1krb9mrzdrc6h0000gn/T/ipykernel_1647/554933739.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  index_pairs     = torch.tensor(index_pairs)


In [20]:
### Skip-gram 모델 준비 작업
from torch import optim

device    = "cuda" if torch.cuda.is_available() else "cpu"
word2vec  = VanilaSkipGram(vocab_size=len(token_to_id), embedding_dim=128).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(word2vec.parameters(), lr=0.1)

In [21]:
### 모델 학습

for epoch in range(10):
    cost = 0.0
    for input_ids, target_ids in dataloader:
        input_ids  = input_ids.to(device)   # center
        target_ids = target_ids.to(device)  # context

        logits = word2vec(input_ids)
        loss   = criterion(logits, target_ids)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 6.197
Epoch :    2, Cost : 5.982
Epoch :    3, Cost : 5.932
Epoch :    4, Cost : 5.902
Epoch :    5, Cost : 5.880
Epoch :    6, Cost : 5.862
Epoch :    7, Cost : 5.847
Epoch :    8, Cost : 5.834
Epoch :    9, Cost : 5.823
Epoch :   10, Cost : 5.812


In [24]:
### 임베딩 값 추출
token_to_embedding = dict()
embedding_matrix = word2vec.embedding.weight.detach().cpu().numpy()

for word, embedding in zip(vocab, embedding_matrix):
    token_to_embedding[word] = embedding 

index = 30
token = vocab[30]
token_embedding = token_to_embedding[token]
print(token)
print(token_embedding)
print(len(token_embedding))

연기
[ 8.3985800e-01  1.0761659e+00 -9.0439522e-01  4.3242589e-02
  8.1296876e-02 -9.7479530e-02 -7.0416003e-01 -9.6324241e-01
 -1.4180402e+00  2.2226593e-01 -2.8250880e+00  1.8537556e-01
 -1.2204744e+00 -3.4485254e-02  7.6117694e-01 -4.1025135e-01
 -2.6833532e-02  2.9363859e-01 -8.7823009e-01  1.0064837e+00
 -8.0417120e-01  8.6418062e-01 -4.3136454e-01 -6.2688684e-01
 -1.0920253e+00  5.6999058e-01  5.2200222e-01 -1.0915482e+00
  4.0858489e-01  1.5445745e+00  4.3164861e-01  8.7255138e-01
  1.2707565e+00  6.8967521e-01 -3.2996199e-01  3.1246388e-02
 -1.7725719e+00  2.0509247e-01  1.8019468e-02 -6.6318232e-01
 -6.9887859e-01 -5.4264057e-01  5.0250506e-01  6.1741930e-01
 -2.2365169e-01  1.0105361e+00  8.0315655e-01  4.5288074e-01
  1.4565660e-01 -6.0031480e-01 -3.8159466e-01  2.6690510e-01
  2.1939619e+00 -1.1505514e+00  3.8696733e-01  2.8555635e-01
 -2.6111897e-02  7.7953339e-01  2.9616660e-01 -1.0148469e-01
 -1.1331215e+00 -1.1230891e+00  3.6436760e-01 -6.2794805e-01
 -1.8235543e-01 -7.21

In [32]:
### 단어 임베딩 유사도 계산
import numpy as np
from numpy.linalg import norm

# 입력 단어와 단언 사전 내의 모든 단어의 코사인 유사도 반환
def cosine_similarity(a, b):
    cosine = np.dot(b, a) / (norm(b, axis=1) * norm(a))
    return cosine

# 가장 가까운 단어 n개 반환
def top_n_index(cosine_matrix, n):
    # 내림차순하여 인덱스 반환
    closest_indexes = cosine_matrix.argsort()[::-1]
    top_n = closest_indexes[1 : n + 1]
    return top_n

# token_embedding[128,1], embedding_matrix[5001,128]
cosine_matrix = cosine_similarity(token_embedding, embedding_matrix)
top_n = top_n_index(cosine_matrix, n=5)

print(f"{token}와 가장 유사한 5 개 단어")
for index in top_n:
    print(f"{id_to_token[index]} - 유사도 : {cosine_matrix[index]:.4f}")

128
640128
연기와 가장 유사한 5 개 단어
및 - 유사도 : 0.2903
되다니 - 유사도 : 0.2816
롱 - 유사도 : 0.2735
파이팅 - 유사도 : 0.2723
만들었다 - 유사도 : 0.2634
